In [1]:
#------------------------------------------Libraries-----------------------

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#------------------------------------------Data Import-----------------------

DF_Final = pd.read_csv('subset.csv')
DF_Final = DF_Final.drop(columns=['Unnamed: 0', 'X'], axis=1)

In [3]:
#------------------------------------------Use TF-IDF to choose dictionary-----------------------
documents = []
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

X = DF_Final.Review
stemmer = WordNetLemmatizer()
for i in range(0, len(X)):
    # Remove all the special characters, like parathesis
    document = re.sub(r'\W', ' ', str(X[i]))
    # remove all single characters: like a, b, c, d
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the startf
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)



from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=0.1, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [5]:
#------------------------------------Negative Sentiment Lexicon-----------------------------------------
f_n = open("negative-words.txt", "r")
Negative_words = f_n.readlines()

NEG = [i.rstrip('\n') for i in Negative_words]

vectorizer = CountVectorizer(vocabulary=np.unique(NEG))
X = vectorizer.fit_transform(documents).toarray()

neg = pd.DataFrame(X)
neg.columns = NEG
neg.to_csv('neg_sentiment.csv')

In [7]:
#------------------------------------Positive Sentiment Lexicon-----------------------------------------
f_p = open("positive-words.txt", "r")
Positive_words = f_p.readlines()

POS = [i.rstrip('\n') for i in Positive_words]


vectorizer = CountVectorizer(vocabulary=np.unique(POS))
X = vectorizer.fit_transform(documents).toarray()

pos = pd.DataFrame(X)
pos.columns = POS
pos.to_csv('pos_sentiment.csv')

In [13]:
#------------------------------------Combining into one dataset-----------------------------------------
num_pos = pos.sum(axis = 1)
num_neg = neg.sum(axis = 1)
joined = pd.DataFrame({'Positive': num_pos, 'Negative': num_neg})
DF_Final = DF_Final.join(joined)
DF_Final.to_csv('data_with_sentiment.csv')